<a href="https://colab.research.google.com/github/juhumkwon/Defense_Cloud/blob/main/CartPole_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gym pygame tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 8.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 118.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 110.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 111.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 116.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.2 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827728 

In [3]:
import gymnasium as gym # Modified: Changed import from gym to gymnasium
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
import random
from collections import deque
import time

# 환경 생성 및 렌더링 모드 지정
env = gym.make("CartPole-v1", render_mode="human") # Modified: Removed new_step_api=True
num_states = env.observation_space.shape[0]
num_actions = env.action_space.n

# 모델 정의
def build_model():
    model = models.Sequential([
        layers.Dense(24, activation='relu', input_shape=(num_states,)),
        layers.Dense(24, activation='relu'),
        layers.Dense(num_actions, activation='linear')
    ])
    return model

model = build_model()
target_model = build_model()
target_model.set_weights(model.get_weights())

# 하이퍼파라미터
gamma = 0.99
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.995
batch_size = 64
memory = deque(maxlen=2000)
optimizer = optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.MeanSquaredError()

# 학습 루프
for ep in range(300):
    state, info = env.reset()
    state = np.reshape(state, [1, num_states]) # 신경망 모델(특히 TensorFlow, Keras 모델)은 입력 데이터를 보통 (배치 크기, 입력 차원) 형태로 받습니다.
    total_reward = 0

    for t in range(500):
        # ✅ 시각화
        env.render()
        time.sleep(0.02)  # 50 FPS 정도로 느리게 보여줌

        # 행동 선택
        if np.random.rand() < epsilon:
            action = random.randint(0, num_actions - 1)
        else:
            q_vals = model(state, training=False).numpy()[0]
            action = np.argmax(q_vals)

        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        next_state = np.reshape(next_state, [1, num_states])
        memory.append((state, action, reward, next_state, done))
        state = next_state # 에이전트가 환경에서 행동(action)을 취한 후 환경으로부터 받은 next_state를 다음 타임스텝의 상태로 사용하기 위해 바꾸는 것입니다.


        total_reward += reward

        if done:
            break

        # 학습
        if len(memory) >= batch_size:
            minibatch = random.sample(memory, batch_size)
            states = np.vstack([x[0] for x in minibatch])
            actions = [x[1] for x in minibatch]
            rewards = [x[2] for x in minibatch]
            next_states = np.vstack([x[3] for x in minibatch])
            dones = [x[4] for x in minibatch]

            """
            만약 같은 네트워크를 현재 상태와 다음 상태 모두에 사용하면,
            네트워크의 파라미터가 학습 도중 계속 바뀌기 때문에
            타겟 값이 계속 변하는 moving target problem이 발생합니다.
            """
            q_targets = model.predict(states, verbose=0)
            q_next = target_model.predict(next_states, verbose=0)

            for i in range(batch_size):
                q_targets[i][actions[i]] = rewards[i] if dones[i] else rewards[i] + gamma * np.max(q_next[i])

            with tf.GradientTape() as tape:
                q_preds = model(states, training=True)
                loss = loss_fn(q_targets, q_preds)
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))


    # 타겟 네트워크 업데이트
    if ep % 20 == 0:
        target_model.set_weights(model.get_weights())

    # Epsilon 감소
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

    if (ep + 1) % 50 == 0:
        print(f"Episode {ep+1}, Reward: {total_reward}, Epsilon: {epsilon:.3f}")

env.close()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Episode 50, Reward: 11.0, Epsilon: 0.778
Episode 100, Reward: 80.0, Epsilon: 0.606
Episode 150, Reward: 211.0, Epsilon: 0.471
Episode 200, Reward: 219.0, Epsilon: 0.367
Episode 250, Reward: 152.0, Epsilon: 0.286
Episode 300, Reward: 131.0, Epsilon: 0.222


In [2]:
!pip install gymnasium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 12.3 MB/s eta 0:00:00
